In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mglearn
%matplotlib inline
import seaborn as sns
import platform
from matplotlib import font_manager , rc

if platform.system() == 'Darwin':
  rc('font' , family = 'AppleGothic')
elif platform.system() == 'Windows':
  path = 'C:/Windows/Fonts/malgun.ttf'
  font_name = font_manager.FontProperties(fname = path).get_name()
  rc('font' , family = font_name)
else:
  print('모름')
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score , precision_score , recall_score , roc_auc_score , f1_score , confusion_matrix , roc_curve , precision_recall_curve

# 교차검증과 그리드 서치

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [3]:
data = wine.iloc[:,:-1].to_numpy()
target = wine.iloc[:,-1].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split

train_input , test_input , train_target , test_target = train_test_split(data , target , random_state = 42 , test_size = 0.2)

In [5]:
#훈련세트를 다시 훈련과 테스트로 나눔.
sub_input , val_input , sub_target , val_target = train_test_split(train_input , train_target , test_size = 0.2 , random_state = 42)

In [6]:
sub_input.shape , val_input.shape

((4157, 3), (1040, 3))

In [7]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input , sub_target)
print(dt.score(sub_input , sub_target))
print(dt.score(val_input , val_target))

0.9971133028626413
0.864423076923077


- 과대적합

## K-Fold 교차검증
 - cross_validate() 교차검증 함수 사용 , 기본값 cv = 5

In [8]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt , train_input , train_target)
#fit_time , score_time , test_score 3개 반영됨.
scores

{'fit_time': array([0.00930285, 0.00697494, 0.00796843, 0.00797343, 0.00797343]),
 'score_time': array([0.0009973, 0.0009985, 0.       , 0.       , 0.       ]),
 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

In [9]:
np.mean(scores['test_score'])

0.855300214703487

- train_test_split 함수는 훈련세트를 분리할 때 섞어서 분리하지만 , cross_validate 함수는 훈련세트를 섞어서 폴드를 나누지 않는다.
- 회귀모델 KFold 분할기 , 분류모델 stratifiedKFold 분할기 사용

In [10]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt , train_input , train_target , cv =  StratifiedKFold())
#fit_time , score_time , test_score 3개 반영됨.
np.mean(scores['test_score'])

0.855300214703487

- 10개로 섞기

In [11]:
from sklearn.model_selection import StratifiedKFold
splitter = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = 42) 
scores = cross_validate(dt , train_input , train_target , cv =  splitter)
#fit_time , score_time , test_score 3개 반영됨.
np.mean(scores['test_score'])

0.8574181117533719

## 그리드 서치
 - 모델의 성능을 가장 높게 하는 최적의 하이퍼파라미터를 찾는 방법
 - cv - result_ : 파라미터 조합별 평가 결과를 dict로 변환
 - best_params_ : 가장 좋은 성능을 낸 파라미터 조합을 반환
 - best_estimator_ : 가장 좋은 성능을 낸 모델 반환
 - best_score_ : 가장 좋은 점수 반환
 
 - 하이퍼파라미터의 값들이 많아지면 많은 시간이 소요
  - 1번 : 10개 , 2번 : 5개 , 3번 : 5개
  - 10 * 5 * 5 = 250번(곱적용)

In [15]:
from sklearn.model_selection import GridSearchCV

# min_impurity_decrease : 노드를 나누기 위한 최소 샘플 수
# max_depth : 트리의 깊이
# min_samples_split : 노드를 분리하기 위한 최소 샘플 수

params = {'min_impurity_decrease' : [0.0001 , 0.0002 , 0.0003 , 0.0004 , 0.0005]}



In [17]:
#n_jobs : 병렬 실행에 사용할 CPU 코어수를 지정하는 것( -1 : 모든 코어를 다 사용하겠다는 뜻)
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42) , params , n_jobs = -1)
gs.fit(train_input , train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [19]:
dt = gs.best_estimator_
dt

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)

In [21]:
print(dt.score(train_input , train_target))
print(dt.score(test_input , test_target))

0.9615162593804117
0.8653846153846154


In [22]:
gs.best_params_

{'min_impurity_decrease': 0.0001}

In [34]:
gs.cv_results_['params']

[{'min_impurity_decrease': 0.0001},
 {'min_impurity_decrease': 0.0002},
 {'min_impurity_decrease': 0.0003},
 {'min_impurity_decrease': 0.0004},
 {'min_impurity_decrease': 0.0005}]

In [35]:
params = {
    'max_depth' : range(5,20),
    'min_samples_split' : range(2,100,10),
    'min_impurity_decrease' : np.arange(0.0001 , 0.001 , 0.0001)
}

In [36]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42) , params , n_jobs = -1)
gs.fit(train_input , train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [38]:
gs.cv_results_['mean_test_score']

array([0.85780355, 0.85799604, 0.85799604, ..., 0.86126601, 0.86165063,
       0.86357629])

In [39]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
best_index

841

In [40]:
gs.cv_results_['params'][best_index]

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

## 랜덤 서치( Random Search)
- 매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포 객체를 전달
- from scipy.stats import uniform , randint
- uniform : 균등분포 샘플링 , rvs()
- randint : rvs()
- 테스트 세트의 점수는 검증세트에 대한 점수보다 조금 낮은 것이 일반적이다.

In [42]:
from scipy.stats import uniform , randint

In [48]:
rgen = randint(0,10)
rgen.rvs(10)

array([9, 1, 5, 1, 7, 5, 1, 8, 0, 1], dtype=int64)

In [52]:
np.unique(rgen.rvs(1000) , return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([ 90,  90,  94, 111, 105, 100,  96, 111,  97, 106], dtype=int64))

In [53]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.85334757, 0.01653354, 0.42749422, 0.40603878, 0.49908369,
       0.33364552, 0.77736354, 0.20079742, 0.13751018, 0.03773595])

In [54]:
params = {
    'max_depth' : randint(20,50),
    'min_samples_split' : randint(2,25),
    'min_impurity_decrease' : uniform(0.0001 , 0.001),
    'min_samples_leaf' : randint(1,25)
}

In [58]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42) , params , n_iter = 100 , n_jobs = -1 , random_state = 42)
rs.fit(train_input , train_target)

print(rs.score(train_input , train_target))
print(rs.score(test_input , test_target))

0.8928227823744468
0.86


In [59]:
rs.best_params_

{'max_depth': 39,
 'min_impurity_decrease': 0.00034102546602601173,
 'min_samples_leaf': 7,
 'min_samples_split': 13}

In [ ]:
np.max(rs.cv_results_['mean_test_score'])

In [62]:
dt = rs.best_estimator_

In [63]:
dt.score(test_input , test_target)

0.86